# Modulación OFDM en Tiempo Discreto

## 1. Introducción

**Orthogonal Frequency Division Multiplexing (OFDM)** es una técnica de modulación digital que divide la información en múltiples subportadoras ortogonales, lo que permite transmitir datos a altas tasas de manera robusta ante interferencias y desvanecimientos selectivos en frecuencia. La utilización de la transformada inversa de Fourier (IFFT) permite generar la señal OFDM en el dominio del tiempo a partir de los símbolos modulados (por ejemplo, QPSK, QAM), facilitando su implementación en sistemas digitales.

## 2. Transmisión OFDM en Tiempo Discreto

El sistema OFDM en tiempo discreto se basa en dividir el flujo de datos en multiples subportadoras ortogonales, transmitiendo simultaneamente cada flujo de datos a traves de una subportadora diferente.
Para una mayor comprension de el proceso de transmision, decidimos separarlo en las siguientes etapas:

### Etapa N° 1: Conversion de Datos a Simbolos

Los datos de entrada (una cadena de numeros binarios) se agrupan en bloques de longitud fija y dependiendo de la técnica de modulación utilizada (QAM o PSK), cada grupo de bits se convierte en un símbolo complejo.

In [1]:
#Generamos una secuencia de bits binarios aleatorios que simulan los datos a transmitir.
import numpy as np

# Parámetros
num_bits = 1024  # Cantidad de bits a transmitir

# Generación de datos binarios aleatorios
data_bits = np.random.randint(0, 2, num_bits)

# Mostrar los primeros 20 bits generados
print("Bits generados (primeros 20):", data_bits[:20])

#Esto simula el flujo de entrada digital. Usamos numpy para generar una secuencia de ceros y unos.

Bits generados (primeros 20): [0 0 0 1 0 1 1 1 0 1 0 0 1 1 1 0 1 0 0 0]


In [3]:
def bits_to_qam_symbols(bits):
    # Asegurarse de que la cantidad de bits sea par
    if len(bits) % 2 != 0:
        bits = np.append(bits, 0) # si es impar le agrega un 0 al final para que sea par y que no genere errores

    # Mapeo 4-QAM estándar (Gray coding)
    # Es un diccionario de mapeo, Cada par de bits se asocia a un número complejo que representa un punto en el plano IQ.
    mapping = {
        (0, 0): -1 + 1j,
        (0, 1): 1 + 1j,
        (1, 1): 1 - 1j,
        (1, 0): -1 - 1j
    }

    # Agrupar bits en pares y mapear a símbolos
    symbols = []
    for i in range(0, len(bits), 2): #Recorre los bits de a dos
        pair = (bits[i], bits[i+1]) #Crea un par de bits como tupla
        symbols.append(mapping[pair]) #Usa el diccionario mapping para convertir ese par en un símbolo complejo

    return np.array(symbols)

# Aplicar la función a los bits generados
qam_symbols = bits_to_qam_symbols(data_bits)

# Mostrar los primeros símbolos
print("Símbolos QAM (primeros 10):", qam_symbols[:10])


Símbolos QAM (primeros 10): [-1.+1.j  1.+1.j  1.+1.j  1.-1.j  1.+1.j -1.+1.j  1.-1.j -1.-1.j -1.-1.j
 -1.+1.j]


#### Modulacion QAM: 

Esta modulacion nos permite enviar más información a través de una sola señal, ya que se usan diferentes combinaciones de amplitudes y fases. Cuantos más niveles de amplitud y fase se utilicen, más datos se pueden transmitir en el mismo espacio de tiempo, pero también se hace más difícil para el receptor descifrar la señal si hay ruido o interferencia.

La ecuación de la función QAM es:

$$s(t) = I(t)\cdot\cos(2\pi f_ct)+Q(t)\cdot\sin(2\pi f_ct)$$

Donde:

- $I(t)$ es la componente en fase *(in-phase)* respecto a la portadora.
- $Q(t)$ es la componente en cuadratura *(quadrature)* respecto a la portadora (desfasada 90°).
- $f_c$ es la frecuencia de la portadora.

Para una primera instancia utilizaremos una modulacion 4-QAM (4 símbolos) esta tiene 2 bits por símbolo.

**Ejemplo:**

Si los bits de entrada son `01`, el simbolo complejo resultante sería $X_k = 1 + j$ utilizando un mapeo estandar (de izquierda a derecha de arriba para abajo).


### Estapa N°2: Mapeo de Simbolos a Subportadoras

Esta etapa consiste en distribuir cada símbolo complejo generado en la etapa anterior en diferentes subportadoras ortogonales. El objetivo es aprovechar el uso de multiples portadoras para transmitir la informacion de manera eficiente.

- **Asignacion de subportadoras:** Los simbolos se agrupan en bloques de longitud $N$, donde $N$ representa el numero de subportadoras.
- **Distribucion en frecuencia:** Cada simbolo se asocia a una subportadora de frecuencia diferente. Esto significa que si tenemos un ancho de banda total de $B$, el espaciado entre subportadoras es $\frac{B}{N}.$

In [4]:
def map_symbols_to_subcarriers(symbols, N):
    # Paso 1: Verificar si la cantidad de símbolos es múltiplo de N
    padding = (-len(symbols)) % N #calcula cuántos ceros hay que agregar si los símbolos no se dividen justo entre N.
    if padding > 0:
        symbols = np.append(symbols, [0]*padding)  # Relleno con ceros si es necesario

    # Paso 2: Agrupar los símbolos en bloques de N
    blocks = symbols.reshape((-1, N))
    return blocks


In [5]:
# Aplicación con N subportadoras
N = 4  # Podés cambiar este valor según tu diseño
symbol_blocks = map_symbols_to_subcarriers(qam_symbols, N)

# Mostrar todos los bloques de símbolos
for i, bloque in enumerate(symbol_blocks):
    print(f"Bloque {i+1}: {bloque}")



Bloque 1: [-1.+1.j  1.+1.j  1.+1.j  1.-1.j]
Bloque 2: [ 1.+1.j -1.+1.j  1.-1.j -1.-1.j]
Bloque 3: [-1.-1.j -1.+1.j  1.+1.j  1.-1.j]
Bloque 4: [-1.-1.j  1.-1.j  1.+1.j  1.-1.j]
Bloque 5: [ 1.-1.j  1.-1.j -1.-1.j  1.-1.j]
Bloque 6: [ 1.-1.j  1.-1.j -1.+1.j -1.+1.j]
Bloque 7: [-1.-1.j -1.+1.j -1.+1.j  1.+1.j]
Bloque 8: [-1.+1.j -1.-1.j -1.-1.j -1.-1.j]
Bloque 9: [ 1.-1.j  1.-1.j  1.-1.j -1.+1.j]
Bloque 10: [-1.-1.j  1.-1.j -1.-1.j -1.+1.j]
Bloque 11: [ 1.+1.j -1.-1.j  1.-1.j  1.-1.j]
Bloque 12: [-1.-1.j  1.-1.j  1.+1.j -1.-1.j]
Bloque 13: [ 1.-1.j -1.+1.j -1.+1.j -1.+1.j]
Bloque 14: [ 1.-1.j  1.+1.j  1.-1.j -1.+1.j]
Bloque 15: [ 1.-1.j  1.-1.j  1.-1.j -1.+1.j]
Bloque 16: [ 1.+1.j -1.+1.j -1.-1.j -1.-1.j]
Bloque 17: [ 1.-1.j  1.-1.j  1.-1.j -1.-1.j]
Bloque 18: [-1.+1.j  1.+1.j -1.+1.j -1.+1.j]
Bloque 19: [ 1.-1.j  1.+1.j  1.-1.j -1.+1.j]
Bloque 20: [ 1.-1.j  1.+1.j -1.-1.j -1.+1.j]
Bloque 21: [-1.-1.j  1.-1.j  1.+1.j  1.-1.j]
Bloque 22: [-1.-1.j  1.-1.j  1.-1.j  1.-1.j]
Bloque 23: [1.+1.j 

### Etapa N°3: Modulacion OFDM mediante IFFT.

Se aplica la Transformada Inversa de Fourier Discreta al bloque de Simbolos, para obtener la señal en dominio temporal.

La señal transmitida en tiempo discreto se expresa como:

$$s(n)=\sum_{k=0}^{N-1}{X_k\cdot e^{\frac{j2\pi kn}{N}}}$$

Donde:

- $X_k$ es el simbolo modulado (cada par dentro del bloque)
- $N$ es el numero total de portadoras.
- $n$ es la cantidad de bloques.

In [ ]:
def apply_ifft_to_blocks(blocks):
    # Aplica la IFFT a cada bloque de símbolos
    time_domain_signals = np.fft.ifft(blocks, axis=1)
    #np.fft.ifft aplica la transfotmada inversa de fourier discreta, el axis = 1 significa que se aplica a cada fila, es decir a cada bloque individual
    return time_domain_signals
    # Devuelve el array de señales temporales. Cada fila representa una señal OFDM lista para ser transmitida.

#en terminos fisicos -> cada simbolo se convierte en una onda y todas las ondas se suman para formar la señal OFDM

# Aplicar la IFFT
ofdm_signals = apply_ifft_to_blocks(symbol_blocks)

# Mostrar las primeras señales temporales
for i, señal in enumerate(ofdm_signals):
    print(f"Señal OFDM bloque {i+1}: {señal}")


Señal OFDM bloque 1: [ 0.5+0.j  0.5-1.j  0.5+0.j -0.5+0.j]
Señal OFDM bloque 2: [0.+0.j 1.+0.j 0.+0.j 0.+1.j]
Señal OFDM bloque 3: [ 0.+0.j -1.+0.j  0.+0.j  0.-1.j]
Señal OFDM bloque 4: [0.+0.j 1.+1.j 0.+0.j 0.+0.j]
Señal OFDM bloque 5: [ 0.+0.j  0.-1.j  0.+0.j -1.+0.j]
Señal OFDM bloque 6: [ 0.5-0.5j -1. +0.j  -0.5-0.5j  0. +0.j ]
Señal OFDM bloque 7: [0. +0.5j 0.5-1.j  0. -0.5j 0.5+0.j ]
Señal OFDM bloque 8: [-0.5+0.j   1. -0.5j  0.5+0.j   0. -0.5j]
Señal OFDM bloque 9: [0.5+0.j  0. -0.5j 0.5+1.j  0. +0.5j]
Señal OFDM bloque 10: [0.+0.j 0.+1.j 0.+0.j 1.+0.j]
Señal OFDM bloque 11: [0.5+1.j  0. +0.5j 0.5+0.j  0. -0.5j]
Señal OFDM bloque 12: [ 0.-0.5j  0.-0.5j  0.-0.5j -1.+0.5j]
Señal OFDM bloque 13: [ 0.5+0.5j -1. +0.j  -0.5+0.5j  0. +0.j ]
Señal OFDM bloque 14: [0. -1.j  0.5+0.5j 0. +0.j  0.5-0.5j]
Señal OFDM bloque 15: [0. -0.5j 0.5+0.j  0. +0.5j 0.5+1.j ]
Señal OFDM bloque 16: [0.+0.5j 1.-0.5j 0.+0.5j 0.+0.5j]
Señal OFDM bloque 17: [ 0.5+0.j  -1. +0.5j -0.5+0.j   0. +0.5j]
Señal OFD

### Etapa N°4: Adición del Prefijo Cíclico (CP)

Para evitar Interferencia Intersimbolo (ISI), se añade un prefijo ciclico al comienzo de cada simbolo OFDM. El prefijo cíclico es una copia de la última parte del símbolo OFDM.

### Ejemplo Conceptual:

Suponiendo el siguiente flujo de datos: `01001011`.

- **Etapa 1:** Los simbolos resultantes del flujo de datos son...

    - $X_0(01) = 1+j$
    - $X_1(00) = -1+j$
    - $X_2(10) = -1-j$
    - $X_3(11) = 1-j$

- **Etapa 2:** Suponiendo $N=4$ subportadoras...

    - $X_0 \rightarrow N_0$
    - $X_1 \rightarrow N_1$
    - $X_2 \rightarrow N_2$
    - $X_3 \rightarrow N_3$

- **Etapa 3:** La señal OFDM resultante será...

$$s(n)=(1+j)e^{j0}+(-1+j)e^{j\frac{2\pi n}{4}}+(-1-j)e^{j\frac{4\pi n}{4}}(1-j)e^{j\frac{6\pi n}{4}}$$

- **Etapa 4:** Suponiendo un prefijo ciclico de longitud 2 y 4 muestras...

    -  $[s(0),s(1),s(2),s(3)]\rightarrow[s(2),s(3),s(0),s(1),s(2),s(3)]$